In [1]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Input, Dense, Activation,Dropout
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


In [2]:
#Folder for the dataset
datasetFolder = '/home/carnd/dbpedia2016/step_2x125/dataset/'

#Number of files
numberOfFiles = 638

#Test split
testSplit=0.1
validationSplit=0.2

In [3]:
def load_data(datasetFolder, datasetXFile, datasetYFile, wrap=True, printIt=False):
    #print('Loading X')
    # load file
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()
    if(printIt):
        print('Input Features: {} x {}'.format(numberOfRows,numberOfCols))
    if(wrap==True):
        maxY = 8384
    else:
        maxY = numberOfCols-1
    half=(numberOfCols//maxY)*0.5
    dataX = np.zeros([numberOfRows,maxY],np.int8)
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(',')
            for i in range(1, len(row)):
                if(int(row[i])<=0):
                    continue;
                val = 1 + ((int(row[i])-1)//maxY);
                if(val>half):
                    val = 0 - (val - half)
                dataX[rowCounter][(int(row[i])-1)%maxY]= val
                #if((1 + ((int(row[i])-1)//maxY))>1):
                #    print("{} data[{}][{}] = {}".format(int(row[i])-1, rowCounter,(int(row[i])-1)%maxY,1 + ((int(row[i])-1)//maxY)))
            rowCounter+=1
        f.close()
   
    #print('Loading Y')
    # load file
    with open(datasetFolder + datasetYFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()

    if(printIt):
        print('Output Features: {} x {}'.format(numberOfRows,numberOfCols))
    dataY = np.zeros([numberOfRows,(numberOfCols-1)],np.float16)
    with open(datasetFolder + datasetYFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(',')
            for i in range(1, len(row)):
                if(int(row[i])<=0):
                    continue;
                dataY[rowCounter][(int(row[i])-1)]=1
            rowCounter+=1
        f.close()
        

    return dataX, dataY

In [4]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv', printIt=True)

Input Features: 3825 x 159766
Output Features: 3825 x 526


In [5]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

In [6]:
print(dataX.shape)
print(dataX[0:5])

(3825, 8384)
[[1 1 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [7]:
print(dataY.shape)
print(dataY[0:5])

(3825, 525)
[[ 0.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [8]:
print("Input Features for classification: {}".format(dataX.shape[1]))
print("Output Classes for classification: {}".format(dataY.shape[1]))

Input Features for classification: 8384
Output Classes for classification: 525


In [9]:
deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(1024, init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(768, init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(512, init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(256, init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_normal'))

In [10]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

def fBetaScore(y_true, y_pred, beta):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = (1+(beta*beta)) * precision * recall / ((beta*beta*precision)+recall+ K.epsilon())
    return f1score


deepModel.compile(loss='binary_crossentropy', optimizer='nadam', metrics=[f1score])

In [11]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX, dataY, nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX, dataY)

In [12]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [13]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-(testSplit+validationSplit))*numberOfFiles)
testSplitIndex=int((1-(testSplit))*numberOfFiles)


numberOfEons = 8
    
for eon in range(0, numberOfEons):
    print('{}. Eon {}/{}'.format(eon+1,eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        #print('Model = {}'.format(model.name))
        deepModel.fit(dataX, dataY, nb_epoch=1, verbose=0, batch_size=256)
        print('Learning deep model for file {} / {} : datasetX/Y_{}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]), end='\r')
        #sc=deepModel.test_on_batch(dataX,dataY)
        #loss = sc[0]
        #f1score = sc[1]
        #loss, f1score=fit_data(deepModel,dataX, dataY)
        #print('Learning deep model for file {} / {} : datasetX/Y_{} loss={:.4f}  f1score={:.4f}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex], loss, f1score), end='\r')

    counts = {} 
    counts[deepModel.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    for testIndex in range(splitIndex, testSplitIndex):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        predY=deepModel.predict_on_batch(dataX)
        true_positives, predicted_positives, possible_positives = countPredictions(dataY, predY)
        counts[deepModel.name]['true_positives'] += true_positives
        counts[deepModel.name]['predicted_positives'] += predicted_positives
        counts[deepModel.name]['possible_positives'] += possible_positives
        print ('Validating deep model {} / {} : - true +ve:{}  pred +ve:{} possible +ve:{}'.format(testIndex+1, testSplitIndex, true_positives,predicted_positives,possible_positives), end='\r')

    count = counts[deepModel.name]
    precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
    recall = (count['true_positives'])/(count['possible_positives']+0.0001)
    f1score = 2.0 * precision * recall / (precision+recall+0.0001)
    print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(deepModel.name, f1score, precision, recall))

counts = {} 
counts[deepModel.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
for testIndex in range(testSplitIndex, numberOfFiles):
    dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
    predY=deepModel.predict_on_batch(dataX)
    true_positives, predicted_positives, possible_positives = countPredictions(dataY, predY)
    counts[deepModel.name]['true_positives'] += true_positives
    counts[deepModel.name]['predicted_positives'] += predicted_positives
    counts[deepModel.name]['possible_positives'] += possible_positives
    print ('Testing deep model {} / {} : - true +ve:{}  pred +ve:{} possible +ve:{}'.format(testIndex+1, numberOfFiles, true_positives,predicted_positives,possible_positives), end='\r')

count = counts[deepModel.name]
precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
recall = (count['true_positives'])/(count['possible_positives']+0.0001)
f1score = 2.0 * precision * recall / (precision+recall+0.0001)
print(' - Final Test Score for {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(deepModel.name, f1score, precision, recall))

1. Eon 1/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8510	 precision = 0.8957 	 recall = 0.8107
2. Eon 2/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8598	 precision = 0.8986 	 recall = 0.8242
3. Eon 3/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8627	 precision = 0.8993 	 recall = 0.8290
4. Eon 4/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8644	 precision = 0.9031 	 recall = 0.8290
5. Eon 5/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8657	 precision = 0.9023 	 recall = 0.8321
6. Eon 6/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8668	 precision = 0.9003 	 recall = 0.8358
7. Eon 7/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8663	 precision = 0.9020 	 recall = 0.8335
8. Eon 8/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8668	 precision = 0.8988 	 recall = 0.8371
 - Final Test Score for Deep Model (5 Dense Layers) 	 f1-score = 0.8668	 precision = 0.8987 	 recall = 0.8371


1. Eon 1/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8510	 precision = 0.8957 	 recall = 0.8107
2. Eon 2/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8598	 precision = 0.8986 	 recall = 0.8242
3. Eon 3/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8627	 precision = 0.8993 	 recall = 0.8290
4. Eon 4/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8644	 precision = 0.9031 	 recall = 0.8290
5. Eon 5/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8657	 precision = 0.9023 	 recall = 0.8321
6. Eon 6/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8668	 precision = 0.9003 	 recall = 0.8358
7. Eon 7/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8663	 precision = 0.9020 	 recall = 0.8335
8. Eon 8/8
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8668	 precision = 0.8988 	 recall = 0.8371
 - Final Test Score for Deep Model (5 Dense Layers) 	 f1-score = 0.8668	 precision = 0.8987 	 recall = 0.8371